## _Medical Insurance Costs_

Pada kasus ini, terdapat data tentang informasi kesehatan dan biaya yang harus dikeluarkan oleh asuransi kesehatan. Infomasi terkait dengan data _medical insurance cost_ adalah sebagai berikut,

1. Age: Usia penerima manfaat
2. Sex: Gender penerima manfaat (_male_, _femele_)
3. Bmi : Body Mass Index
4. Children: Jumlah anak/tanggungan yang dicover oleh pihak asuransi
5. Smoker: Status perokok (_yes_, _no_)
6. Region: Wilayah tempat tinggal penerima manfaat
7. Charges: Biaya yang dikeluarkan oleh asuransi

In [48]:
# Selayang pandang data Medical Insurance Costs
import numpy as np
import pandas as pd

df = pd.read_csv('data/insurance.csv')

display(df.head())

display(df.corr())

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.900,0,yes,southwest,16884.92400
1,18,male,33.770,1,no,southeast,1725.55230
2,28,male,33.000,3,no,southeast,4449.46200
3,33,male,22.705,0,no,northwest,21984.47061
4,32,male,28.880,0,no,northwest,3866.85520


,age,bmi,children,charges
age,1.000000,0.109272,0.042469,0.299008
bmi,0.109272,1.000000,0.012759,0.198341
children,0.042469,0.012759,1.000000,0.067998
charges,0.299008,0.198341,0.067998,1.000000


### Tantangan

Buatlah model regresi untuk memprediksi biaya yang harus dikeluarkan oleh pihak asuransi berdasarkan data. Validasi performa model regresi Anda dengan nilai ***R-squared ($R^2$)***

#### _Tasks_

1. Pastikan semua variabel kategorial diolah dengan baik. (Gunakan fitur mapping pada pandas)
2. Cek kondisi multicollinearity untuk semua variabel independen. Jika ada, antar variabel apakah itu?
3. Pastikan model menggunakan variabel yang tidak memiliki nilai multicollinearity yang tinggi
4. (Hints) Anda dapat menggunakan nilai ***Variance Inflation Factor (VIF)*** untuk mengetahui tingkat multicollinearity pada sebuah variabel independent.
5. Evaluasi model yang Anda buat dengan nilai $R^2$
6. Simpulkan, variabel independen apa saja yang dapat digunakan untuk menghasilkan model regresi yang baik pada kasus _medical insurance costs_?

#### (Hints) Interpretasi Nilai VIF

- 1 - variabel indenpenden tidak memiliki korelasi dengan variabel independen yang lain
- 1 < VIF < 5 - variabel independen sedikit memiliki korelasi dengan variabel independen yang lain
- VIF > 5 - variabel independen memiliki korelasi yang kuat dengan variabel independen lainnya
- VIF > 10 - variabel independen miliki korelasi yang sangat kuat dengan variabel independen dan perlu diperhatikan lebih lanjut

#### (Hints) Implementasi Perhitungan VIF

VIF dapat dihitung secara langsung dengan menggunakan library dari `statsmodels`

#### (Hints) Scatterplot Korelasi Antar Variabel

![var_cor](assets/var_corr.png)

In [49]:
print("Nama : R.Muhammad Azmi Herdi Shofiyullah")
print("Kelas : TI-3A")
print("Absen : 21")

Nama : R.Muhammad Azmi Herdi Shofiyullah
Kelas : TI-3A
Absen : 21


In [50]:
import matplotlib.pyplot as plt 
import pandas as pd

df = pd.read_csv('data/insurance.csv')

#cek variasi data
print(df['region'].value_counts())
print('\n')
print(df['sex'].value_counts())
print('\n')
print(df['smoker'].value_counts())
print('\n')

southeast    364
northwest    325
southwest    325
northeast    324
Name: region, dtype: int64


male      676
female    662
Name: sex, dtype: int64


no     1064
yes     274
Name: smoker, dtype: int64




In [51]:
#ubah data menjadi angka
new_labels = {
    'southeast': 1,
    'northwest': 2,
    'southwest': 3,
    'northeast': 0
}

gender = {
    'male': 1,
    'female': 0
}

smoke = {
    'yes': 1,
    'no': 0
}

df['region'] = df['region'].map(new_labels)
df['sex'] = df['sex'].map(gender)
df['smoker'] = df['smoker'].map(smoke)

# Cek data
df.head()

,age,sex,bmi,children,smoker,region,charges
0,19,0,27.900,0,1,3,16884.92400
1,18,1,33.770,1,0,1,1725.55230
2,28,1,33.000,3,0,1,4449.46200
3,33,1,22.705,0,0,2,21984.47061
4,32,1,28.880,0,0,2,3866.85520


In [52]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
  
# the independent variables set
T = df[['age', 'sex', 'children', 'smoker', 'region', 'bmi']]
  
# VIF dataframe
vif_data = pd.DataFrame()
vif_data["feature"] = T.columns
  
# calculating VIF for each feature
vif_data["VIF"] = [variance_inflation_factor(T.values, i)
                          for i in range(len(T.columns))]
  
print(vif_data)

    feature       VIF
0       age  7.625316
1       sex  2.002391
2  children  1.807247
3    smoker  1.257496
4    region  2.671980
5       bmi  9.649267


In [53]:
#drop kolom yang diperlukan
df = df.drop(df.columns[0], axis=1)
df = df.drop(df.columns[1], axis=1)

df.head()

,sex,children,smoker,region,charges
0,0,0,1,3,16884.92400
1,1,1,0,1,1725.55230
2,1,3,0,1,4449.46200
3,1,0,0,2,21984.47061
4,1,0,0,2,3866.85520


In [54]:
#buat copy tabel
new_df = df
new_df.head()

,sex,children,smoker,region,charges
0,0,0,1,3,16884.92400
1,1,1,0,1,1725.55230
2,1,3,0,1,4449.46200
3,1,0,0,2,21984.47061
4,1,0,0,2,3866.85520


In [55]:
#mulai mencari nilai R2
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score


X = new_df.iloc[:, :-1].values
y = new_df.iloc[:, 4].values

Xm_train, Xm_test, ym_train, ym_test = train_test_split(X, y, test_size=0.2, random_state=100)

y = y.reshape(len(y), 1)
y.shape

(1338, 1)

In [56]:
mlr = LinearRegression()
mlr.fit(Xm_train, ym_train)

ym_pred = mlr.predict(Xm_test)

gabung = np.concatenate((ym_test, ym_pred))
gabung

array([ 1826.843     , 20177.67113   ,  7421.19455   ,  1877.9294    ,
       15518.18025   , 29523.1656    , 13143.86485   ,  3279.86855   ,
       18972.495     , 12950.0712    ,  8083.9198    , 18218.16139   ,
        8334.45755   ,  2727.3951    ,  7441.053     , 38711.        ,
       10338.9316    , 11552.904     ,  4751.07      , 11381.3254    ,
       10226.2842    ,  1526.312     ,  7222.78625   , 21259.37795   ,
       10848.1343    , 58571.07448   ,  3268.84665   , 48517.56315   ,
       21880.82      , 13457.9608    , 12592.5345    ,  2457.21115   ,
       47896.79135   , 33732.6867    ,  6600.361     , 14349.8544    ,
       18963.17192   , 39047.285     ,  3645.0894    , 30942.1918    ,
        7624.63      ,  7650.77375   ,  2498.4144    ,  4949.7587    ,
        2196.4732    ,  3972.9247    ,  4415.1588    ,  2362.22905   ,
       11272.33139   ,  5662.225     , 13405.3903    ,  8269.044     ,
        4915.05985   ,  7729.64575   ,  7045.499     , 11674.13      ,
      

In [57]:
r2_mlr = r2_score(ym_test, ym_pred)
print(r2_mlr)
print("Jawaban Final : ", r2_mlr)

0.6829316358709019
Jawaban Final :  0.6829316358709019


2. Untuk variabel yang merupakan multicolinearity adalah 'age', 'sex', 'children', 'smoker', 'region', 'bmi' yang dimana memiliki nilai VIF yang telah disebutkan di atas.
 ---
6. untuk kesimpulannya adalah variabel yang dibutuhkan untuk mendapatkan nilai yang adalah semua variabel kecuali variabel age dan bmi dibutuhkan untuk mendapatkan nilai terbaik.